# 1. Cargar todos los datos

In [1]:
from cmath import nan
import sentence_transformers
import torch
from sentence_transformers import SentenceTransformer
import numpy as np
import pandas as pd
import nltk
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

# Cargar el BERT de partida

BERTBASE =  'sentence-transformers/stsb-bert-base'
PRITAMDEKAMODEL = 'pritamdeka/BioBERT-mnli-snli-scinli-scitail-mednli-stsb'
bertmodel = SentenceTransformer(PRITAMDEKAMODEL)
# Se puede aumentar max_seq_length?

# Función clean abstract

# Download the stopwords from NLTK
nltk.download('punkt')
nltk.download('stopwords')

cached_stopwords = stopwords.words('english')

def clean_abstract(abstract):
    if isinstance(abstract, float) and np.isnan(abstract):
        return ''
    # Convert the text to lowercase
    abstract = abstract.lower()

    # Remove punctuation
    abstract = abstract.translate(str.maketrans('', '', string.punctuation))

    # Tokenize the text
    tokens = word_tokenize(abstract)

    # Remove stopwords
    tokens = [word for word in tokens if not word in cached_stopwords]

    # Join the tokens back into a single string
    abstract = ' '.join(tokens)

    return abstract

# Obtener los datos de entrenamiento

PATH_DATA = '../pubmed-queries/abstracts'
PATH_DATA_CSV = PATH_DATA + '/abstracts.csv'
PATH_DATA_FENOTIPOS = '../pubmed-queries/results/phenotypes-22-12-15.csv'
PATH_INDEX_FENOTIPOS = PATH_DATA + '/index-phenotypes.csv'
SEED = 42

dfPapers = pd.read_csv(PATH_DATA_CSV, sep='\t', low_memory=False, na_values=['', nan])
dfPhenotypes = pd.read_csv(PATH_DATA_FENOTIPOS, sep=';', low_memory=False, na_values=['', nan])
dfIndex = pd.read_csv(PATH_INDEX_FENOTIPOS, sep='\t', low_memory=False, na_values=['', nan])

# Cargar la ontología

from pyhpo import Ontology

onto = Ontology('../pubmed-queries/hpo-22-12-15-data')

# phenotypeId	phenotypeName	numberPapers	paperList

# Tomar la lista de fenotipos = tags
tags = dfIndex['phenotypeName']
numlabels = len(tags)
print(numlabels, 'tags')
print(tags[:5])

from itertools import combinations
from sentence_transformers import util

# Tomar muestra aleatoria de pares de fenotipos
MARGIN = 0.3743
if MARGIN == 0: # Estimar un margin apropiado
    unique_pairs = combinations(dfIndex['phenotypeName'].drop_duplicates(), 2)
    df_pairs = pd.DataFrame(unique_pairs, columns=['phenotype1', 'phenotype2']).sample(frac=0.2, random_state=SEED)
    print('Unique pairs:', len(df_pairs))
    print('Pair 1:', df_pairs.iloc[0])
    #df_pairs['distance']=df_pairs.apply(lambda x: float(losses.SiameseDistanceMetric.COSINE_DISTANCE(torch.from_numpy(model.encode(x['phenotype1'])), torch.from_numpy(model.encode(x['phenotype2'])))), axis=1)
    df_pairs['distance'] = df_pairs.apply(lambda x: 1-util.cos_sim(model.encode(x['phenotype1']), model.encode(x['phenotype2'])), axis=1)
    margin = min(df_pairs['distance']).numpy()[0][0]
    print('Margin:', margin)

# Separar abstracts en train, validation y test

# quitar NA's en la columna abstract
print('Na\'s:', dfPapers['abstract'].isna().sum())
dfPapers = dfPapers.dropna(subset=['abstract'])

train = dfPapers.sample(frac=0.1, random_state=SEED)
dTest = dfPapers.drop(train.index).sample(frac=0.2, random_state=SEED)
dVal = train.sample(frac=0.2, random_state=SEED)
dTrain = train.drop(dVal.index)
num_examples = len(dTrain)

# Considerar train_test_split

# paperId	phenotypeId	phenotypeName	title	abstract
list_ = [dTrain, dVal, dTest]
names = ['Train', 'Validation', 'Test']
for j in range(0, 3):
    l = list_[j]
    print(names[j],': ', len(l), '\n')
    for i in range(0, 2):
        print(l.iloc[i])
    print('')

from torch.utils.data import DataLoader, Dataset
from sentence_transformers import SentenceTransformer, SentencesDataset, losses, evaluation, InputExample
torch.manual_seed(SEED)

num_epochs = 2

model = bertmodel

mapping = {tag: i for i, tag in enumerate(tags)}


def getLabelNumber(phenotypeName):
    return mapping[phenotypeName]

# TODO: Documentarse cómo se prepara el DataLoader con los pares abstract-fenotipo
# imagino que en el conjunto de train solo se usan los abstracts y en el conjunto de validación y test se usan los abstracts y los fenotipos

print("Preparing dataloaders...")

print('Cleaning abstracts...')
print('example:', clean_abstract(dTrain['abstract'].iloc[0]))

abstractsTrain = [InputExample(texts=[clean_abstract(x)], label=mapping[y]) for x, y in zip(dTrain['abstract'], dTrain['phenotypeName'])]
train_dataloader = DataLoader(abstractsTrain, shuffle=True, batch_size=16)

print('Validation')
pairsVal = [InputExample(texts=[clean_abstract(x)], label=mapping[y]) for x, y in zip(dTrain['abstract'], dVal['phenotypeName'])]
val_dataloader = DataLoader(pairsVal, shuffle=False, batch_size=16)

print('Test')
pairsTest = [InputExample(texts=[clean_abstract(x)], label=mapping[y]) for x, y in zip(dTrain['abstract'], dTest['phenotypeName'])]
test_dataloader = DataLoader(dTest, shuffle=False, batch_size=16)

# TODO: Documentarse sobre loss y evaluator

print("Preparing loss and evaluator...")
soft_loss = losses.SoftmaxLoss(model=model, sentence_embedding_dimension=model.get_sentence_embedding_dimension(), num_labels=numlabels)
# Esta no sirve porque recibe un par de sentencias y un label, no una sentencia y un label
train_loss = losses.BatchAllTripletLoss(model=model, distance_metric=losses.BatchHardTripletLossDistanceFunction.cosine_distance, margin=MARGIN)

evaluator = evaluation.LabelAccuracyEvaluator(val_dataloader, '', softmax_model=soft_loss, write_csv=True)


# TODO: Documentarse sobre los hiperparámetros y preparar el grid

FITTED = True
PATH_TUNED = './output/fine-tuned-bio-bert'
if FITTED:
    print("Loading fitted model...")
    fmodel = SentenceTransformer(PATH_TUNED)
else:
    print("Fitting...")
    fmodel = model.fit(
        train_objectives=[(train_dataloader, train_loss)],
        #evaluator=evaluator,
        epochs=num_epochs,
        #evaluation_steps=4,
        warmup_steps=int(0.25*(num_examples//16)),
        output_path='./output/fine-tuned-bio-bert',
        save_best_model=True,
        checkpoint_path='./checkpoint',
        checkpoint_save_steps=25,
        checkpoint_save_total_limit=5
    )


[nltk_data] Downloading package punkt to /home/domingo/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/domingo/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


100 tags
0                    Temporomandibular joint ankylosis
1                                             Dyslexia
2    Stippling of the epiphysis of the proximal pha...
3                                 Ankle joint effusion
4                              Reduced C-peptide level
Name: phenotypeName, dtype: object
Na's: 1854
Train :  1710 

paperId                                                   18789087
phenotypeId                                             HP:0032217
phenotypeName                                     Indurated nodule
title            Congenital atrophic dermatofibrosarcoma protub...
abstract         Dermatofibrosarcoma protuberans is a rare, mal...
Name: 4591, dtype: object
paperId                                                   11999861
phenotypeId                                             HP:0030553
phenotypeName                    Visual acuity no light perception
title                            Visual marking and visual change.
abstract         Five exp

# 2. Evaluar con las deltas
Este experimento consiste en comparar las distancias en la ontología de pares de fenotipos arbitrarios con las distancias entre sus embeddings (original y finetuneado). Si las distancias se parecen más que antes en cada epoch quiere decir que el modelo ha aprendido algo.  
Algunas consideraciones:
* 1. La similitud de Resnik tiene un rango y la distancia coseno otro.
* 2. La correlación de Pearson puede ayudar más: queremos que cuando una sea baja la otra también.
* 3. Lo ideal sería que exista una isometría entre ambos espacios ($E_1$ la ontología y $E_2\subset\mathbb{R}^{768}$ los embeddings):
$${\displaystyle \exists \varphi :E_{1}\to E_{2} \mid \forall (x,y)\in E_{1}\times E_{1}:\ d_{1}(x,y)=d_{2}(\varphi (x),\varphi (y))}$$
y que la isometría sea precisamente el embedding. Pero esto lo veo muy difícil porque un conjunto es el árbol de la ontología y otro una esfera (la similitud coseno no entiende de tamaños), por lo que las distancias se calculan de manera muy distinta.
* 4. En cualquier caso este experimento hay que realizarlo igual, porque es la única forma de evaluar el modelo que tenemos sin usar un clasificador.  
* 5. Si definimos las siguientes variables aleatorias: $X =$ "distancia en la ontología entre 2 fenotipos escogidos al azar"  
  $Y_1 = $ "distancia entre los embeddings obtenidos con el modelo original"  
  $Y_2 = $ """ finetuneado".  
  Entonces podemos comparar las variables aleatorias $$Z_i = X - Y_i, i = 1,2$$. Si conocemos la distribución de las $Z_{i}$ (ojalá sea normal) podemos hallar un intervalo de confianza al 95% para $Z_i$ y también hacer un contraste para ver si $$\abs{Z_{2}} < \abs{Z_{1}}$$ que es lo que queremos comprobar.

In [5]:
# Pasos de evaluación
# 1. Extraer una muestra de fenotipos
# 2. Calcular los embeddings de los fenotipos con el modelo original
# 3. Calcular los embeddings de los fenotipos con el modelo entrenado
# 4. Extraer una muestra de pares de fenotipos con sus embeddings
# 5. Para cada muestra calcular
    # a. La distancia entre los embeddings (original y tuneado)
    # b. La distancia en la ontología
# 6. Calcular la correlación entre las distancias a y b
# 7. Calcular cuánto se acerca una distancia a otra (los deltas)
# 8. Obtener los resultados (csv, gráficos)
# 9. Repetir para más epochs

# 1. Extraer una muestra de fenotipos
# primero probemos con los que tenemos en dfIndex, que son los que se usaron para buscar los abstracts
# (validación)
# luego se probará con fenotipos no usados (test)

dmp = pd.DataFrame(dfIndex, columns=['phenotypeName'])

# 2. Calcular los embeddings de los fenotipos con el modelo original
# 3. Calcular los embeddings de los fenotipos con el modelo entrenado

l1 = fmodel.encode(dmp["phenotypeName"])
l2 = bertmodel.encode(dmp["phenotypeName"])
emb = zip(dmp["phenotypeName"], l1, l2)

# Hasta aquí bien

l3 = list(emb)
#print('list of embeddings', len(l3))

#for i in range(0,len(l3)):
    #print(i, l3[i][0])


# 3.5: guardar csv con los embeddings
df_emb = pd.DataFrame(l3, columns=['phenotypeName', 'original', 'tuned'])
df_emb.to_csv(PATH_DATA + '/embeddings.csv', index=False, sep=';')

# 4. Extraer una muestra de pares de fenotipos con sus embeddings
from itertools import combinations

index_pairs = combinations(range(0,len(df_emb)), 2)
#print(len(df_emb))
#print(len(dmp))
#print(index_pairs)
lpairs = []
for (i,j) in index_pairs:
    # extract rows i,j from df_emb
    #print(i,j)
    #print(df_emb.iloc[i], df_emb.iloc[j])
    lpairs.append([df_emb.iloc[i][0], df_emb.iloc[i][1], df_emb.iloc[i][2], df_emb.iloc[j][0], df_emb.iloc[j][1], df_emb.iloc[j][2]])

df_pairs = pd.DataFrame(lpairs, columns=['phenotype1', 'original1', 'tuned1', 'phenotype2', 'original2', 'tuned2'])

#print(df_pairs)
df_pairs.to_csv(PATH_DATA + '/pairs.csv', index=False, sep=';')


/tmp/ipykernel_12673/77673000.py:53: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  lpairs.append([df_emb.iloc[i][0], df_emb.iloc[i][1], df_emb.iloc[i][2], df_emb.iloc[j][0], df_emb.iloc[j][1], df_emb.iloc[j][2]])
